###Set Up

1. Install packages to run **old model**
2. Import all libraries
3. Mount Google Drive
4. Define Country's DF & BASE_PATH
5. Define Dtype for column in DF

In [ ]:
#It can take up to 6 mins
# !pip install scikit-learn==1.2.2 numpy==1.26.4

In [ ]:
!pip install ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.5 MB/s eta 0:00:00


In [ ]:
import nltk
import pandas as pd
import matplotlib.pyplot as plt
from typing import List
from openai import OpenAI
from nltk.tokenize import word_tokenize
from random import choice
import re
from pathlib import Path
import joblib
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from ftfy import fix_encoding
import torch
from datasets import Dataset

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from google.colab import userdata
API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
TARGET_COUNTRY = 'de' # 'be' or 'de'
BASE_PATH = Path('/content/drive/MyDrive/deep-learning/transactions/')
DATASET_PATH = BASE_PATH / 'Datasets-Versions'

In [ ]:
#TODO: fix mixed dtypes
dtype_map = {
    "country": "string",
    "APC1": "string",
    "APC2": "string",
    "language": "string",
    "counterparty_name": "string",
    "communication": "string",
    "cleaned_text_bert": "string",
    "old_model_category": "string",
    "VATType": "category",
    "category_description": "category",
    "category_id": "category",
    "baseCurrencyAmount": "float64"
}

### Parse dataset

1. Filtering DF to Specified Country
2. Defining Regexes
3. Construct 'cleaned_text_bert'
4. Drop NaN observations in 'cleaned_text_bert'
5. Adding description column to categories
6. Formatting input for Train & Inference

In [ ]:
df = pd.read_csv(BASE_PATH / 'datasets' / 'df_final.csv')

In [ ]:
df = df[df['country'] == TARGET_COUNTRY]

In [ ]:
iban_re = re.compile(
    r'\b'                                            # Anchor: Start at a word boundary
    r'(?:'                                           # Start non-capturing group for the two formats
      r'[A-Z]{2}[0-9]{2}'                            # Mandatory start: Country Code (AA) + Check Digits (##)
      r'(?:[\s]?[A-Z0-9]{4}){2,7}'                   # 2 to 7 groups of (optional space + 4 alphanumeric chars)
      r'(?:[\s]?[A-Z0-9]{1,4})?'                     # Optional final group of 1 to 4 chars, optionally preceded by a space
    r'|'                                             # OR
      r'[A-Z]{2}[0-9]{2}[A-Z0-9]{10,30}'             # AA## followed by 10 to 30 contiguous alphanumeric chars
    r')'
    r'\b',                                           # Anchor: End at a word boundary
    flags=re.IGNORECASE                              # Match case-insensitively (most IBAN parsers ignore case on letters)
)

In [ ]:
big_num_re = re.compile(r'\d{5,}')

In [ ]:
dates_re = re.compile(
    r'\d{4}-\d{2}-\d{2}'   # yyyy-mm-dd
    r'|'
    r'\d{4}/\d{2}/\d{2}'   # yyyy/mm/dd
    r'|'
    r'\d{4}\.\d{2}\.\d{2}' # yyyy.mm.dd
    r'|'
    r'\d{2}-\d{2}-\d{4}'   # dd-mm-yyyy
    r'|'
    r'\d{2}/\d{2}/\d{4}'   # dd/mm/yyyy
    r'|'
    r'\d{2}\.\d{2}\.\d{4}' # dd.mm.yyyy
    r'|'
    r'\b\d{2}[-/\.]\d{2}[-/\.]\d{2}\b'      # 02-10-23, 10/02/23, etc.
    r'|'
    r'\b\d{1,2}\s+(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)[a-z]*[\s,]+\d{4}\b'   # 02 Oct 2023, 2 October 2023
    r'|'
    r'\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Sept|Oct|Nov|Dec)[a-z]*\s+\d{1,2},?\s+\d{4}\b'    # Oct 02, 2023, October 2 2023
    r'|'
    r'\b\d{4}-\d{2}-\d{2}[T\s]\d{2}:\d{2}(:\d{2})?(\.\d+)?(Z|[+-]\d{2}:?\d{2})?\b'
)


In [ ]:
iban_re = re.compile(
    r'\b'                                            # Anchor: Start at a word boundary
    r'(?:'                                           # Start non-capturing group for the two formats
      r'[A-Z]{2}[0-9]{2}'                            # Mandatory start: Country Code (AA) + Check Digits (##)
      r'(?:[\s]?[A-Z0-9]{4}){2,7}'                   # 2 to 7 groups of (optional space + 4 alphanumeric chars)
      r'(?:[\s]?[A-Z0-9]{1,4})?'                     # Optional final group of 1 to 4 chars, optionally preceded by a space
    r'|'                                             # OR
      r'[A-Z]{2}[0-9]{2}[A-Z0-9]{10,30}'             # AA## followed by 10 to 30 contiguous alphanumeric chars
    r')'
    r'\b',                                           # Anchor: End at a word boundary
    flags=re.IGNORECASE                              # Match case-insensitively (most IBAN parsers ignore case on letters)
)

In [ ]:
fin_re = re.compile(
    r'\b(?:'
    r'sumup'
    r'|pluxee'
    r'|paypal(?:\s+europe)?(?:\s+s\.?a\.?r\.?l\.?)?(?:\s+et\s+cie)?(?:\s+s\.?c\.?a\.?)?'  # PayPal legal variants
    r'|ing(?:\s+app)?'
    r'|kbc(?:\s+mobile|\s+brussels|\s+betaalknop)?'
    r'|crelan'
    r'|kredbebb\w*'
    r'|bancontact'
    r'|maestro'
    r'|debit\s+mastercard'
    r'|mastercard'
    r'|visa'
    r'|apple\s*pay'
    r'|\bpaypal\b[\s,;\-]*'
    r'|paddlenet\s*[\n,;\-]*\s*(.+)'
    r'|google\s*g\s*suite[\s_+\-]*([\w\s]+)'
    r')\b',
    flags=re.IGNORECASE
)

In [ ]:
punct_re = re.compile(r'[^\w\s\&]')

In [ ]:
def preprocess_text(communication, counterparty_name) -> str:
    """
    Combines communication + counterparty_name, fixes encoding,
    removes big numbers/dates, cleans text for BERT.
    """

    comm = '' if pd.isna(communication) else str(communication)
    cp   = '' if pd.isna(counterparty_name) else str(counterparty_name)

    combined = (cp + ' ' + comm).strip()
    combined = fix_encoding(combined)
    combined = combined.lower()

    combined = fin_re.sub('', combined)
    combined = iban_re.sub('', combined)
    combined = big_num_re.sub('', combined)
    combined = dates_re.sub('', combined)
    combined = url_re.sub('', combined)
    combined = punct_re.sub('', combined)


    tokens = word_tokenize(combined)
    return ' '.join(tokens).strip()

In [ ]:
df['cleaned_text_bert'] = df.apply(lambda row: preprocess_text(row['communication'], row['counterparty_name']),axis=1)

In [ ]:
#Drop NAN in cleaned_text_bert
df.drop(df[df['cleaned_text_bert'] == ''].index, inplace=True)

In [ ]:
readable_labels = pd.read_json(BASE_PATH/'expense-categories'/f'{TARGET_COUNTRY}_en.json')
mapping = dict(zip(readable_labels["category_id"], readable_labels["title"]))
df['category_description'] = df['category_id'].map(mapping)

In [ ]:
def safe_str(x):
    return "" if pd.isna(x) else str(x)

In [ ]:
def build_inputs_format(row):
    input = str("Counter party name and communication of transaction: " + safe_str(row['cleaned_text_bert']) +
                " User profession: " + safe_str(row['APC1']) +
                " Specific user category occupation: " + safe_str(row['APC2']) +
                " Transaction amount: " + safe_str(row['baseCurrencyAmount']))
    return input

In [ ]:
df['input_text'] = df.apply(build_inputs_format, axis=1)

###Truncating

1. Truncating Categories with under 100 obs
2. Truncating Categories to be under 5% of df

In [ ]:
mask = df['category_id'].value_counts()
under_represented = mask[mask < 100]

In [ ]:
#drop under_represented
df = df[~df['category_id'].isin(under_represented.keys())]

In [ ]:
#Truncate OVER-REPRESENTED observations
def cap_class_representation(df, target_column, max_percentage=0.05):
    total_samples = len(df)
    max_samples_per_class = int(total_samples * max_percentage)

    balanced_dfs = []
    for class_label in df[target_column].unique():
        class_df = df[df[target_column] == class_label]
        if len(class_df) > max_samples_per_class:
            class_df = class_df.sample(n=max_samples_per_class, random_state=42)
        balanced_dfs.append(class_df)

    return pd.concat(balanced_dfs, ignore_index=True)


df = cap_class_representation(df, 'category_id')

In [ ]:
df.shape

(625922, 12)

### Old Model baseline

1. Import old model based on Country
2. Run old model on df
3. Remap old labels to new labels

In [ ]:
sentence_transformer = SentenceTransformer('paraphrase-mpnet-base-v2')

In [ ]:
class SimpleClassifier:
    """Lightweight classifier wrapper for Colab"""

    def __init__(self, country_code):
        # Load model and metadata
        model_file = BASE_PATH /'old-models'/f'classifier_{country_code}.joblib'
        metadata_file = BASE_PATH /'old-models'/f'classifier_{country_code}_metadata.json'

        self.model = joblib.load(model_file)

        with open(metadata_file, 'r') as f:
            self.metadata = json.load(f)

        self.country = country_code
        self.classes = self.metadata['classes']

        print(f"✅ Loaded {country_code.upper()} classifier")
        print(f"   📊 {len(self.classes)} categories")
        print(f"   🔢 {self.metadata['n_features']} features expected")

    def classify_transactions(self, embeddings):
        """Classify transactions and return results with confidence"""
        predictions = self.model.predict(embeddings)
        return predictions[0]

In [ ]:
from tqdm.notebook import tqdm

old_classifier = SimpleClassifier(TARGET_COUNTRY)
val_df = pd.read_csv(BASE_PATH/'Datasets-Versions'/f'{TARGET_COUNTRY}_val.csv')
old_model_predictions = []

for i, obs in tqdm(enumerate(val_df['cleaned_text_bert']), total=len(val_df['cleaned_text_bert'])):
    embeddings = sentence_transformer.encode(str(obs))
    example_embeddings = np.array(embeddings).reshape(1, -1)

    results = old_classifier.classify_transactions(example_embeddings)
    old_model_predictions.append(results)

In [ ]:
val_df['old_model_category'] = old_model_predictions

In [ ]:
category_remap = {
  '615500': '623000',
  '610000a': '610000c',
  '610000b': '610000c','610100': '610100b','610110': '610100b','618040': '618060','612200a': '612201','612200b': '612201','612200c': '612202','612200d': '612202','612200e': '612202','612200f': '612202','612120': '612100','612000': '612000a','231000': '612000a','612150': '612150d','612150a': '612150d','240200a': '612150d','240200b': '612150d','240200c': '612150d','612150b': '612150e','612150c': '612150e','613430': '613430b','613430a': '613430b','613431': '613431b','613431a': '613431b','613432': '613432b','613432a': '613432b','613350': '613350b','613350a': '613350b',
  'de.travel.for_self_employed.drive_to_work': 'de.travel.for_self_employed.car_private_use',
  'de.vehicle.parking': 'de.vehicle.business_car.parking',
  'de.vehicle.fuel': 'de.vehicle.business_car.fuel',
  'de.vehicle.other': 'de.vehicle.business_car.other',
  'de.vehicle.insurance': 'de.vehicle.business_car.insurance',
  'de.vehicle.leasing': 'de.vehicle.business_car.leasing',
  'de.vehicle.road_and_registration_taxes': 'de.vehicle.business_car.road_and_registration_taxes',
  'de.vehicle.repair': 'de.vehicle.business_car.repair',
  'de.vehicle.purchase.car': 'de.vehicle.business_car.purchase',
  'de.vehicle.purchase.truck': 'de.vehicle.business_utility_vehicle.purchase',
  'de.vehicle.purchase.other': 'de.vehicle.business_car.purchase.other',
  'de.office_supplies': 'de.office.supplies',
  'de.categories.mailing.stamps_postage': 'de.office.mailing.stamps_postage',
  'de.categories.mailing.package': 'de.office.mailing.package',
  'de.subcontracting.displayName': 'de.subcontracting',
  'de.goods_and_materials.goods_for_resell.zero_percent': 'de.goods_and_materials.goods_for_resell',
  'de.goods_and_materials.goods_for_resell.seven_percent':'de.goods_and_materials.goods_for_resell',
  'de.goods_and_materials.goods_for_resell.nineteen_percent':'de.goods_and_materials.goods_for_resell',
  'de.goods_and_materials.raw_material.zero_percent': 'de.goods_and_materials.raw_material',
  'de.goods_and_materials.raw_material.seven_percent': 'de.goods_and_materials.raw_material',
  'de.goods_and_materials.raw_material.nineteen_percent': 'de.goods_and_materials.raw_material',
  'de.investments.office_furniture.below_250': 'de.office.furniture',
  'de.investments.office_furniture.between_250_and_800': 'de.office.furniture',
  'de.investments.office_furniture.above_800': 'de.office.furniture',
  'de.technology.hardware.below_250': 'de.technology.hardware',
  'de.technology.hardware.between_250_and_800': 'de.technology.hardware',
  'de.technology.hardware.above_800': 'de.technology.hardware',
  'de.investments.tools.below_250': 'de.investments.tools',
  'de.investments.tools.between_250_and_800': 'de.investments.tools',
  'de.investments.tools.above_800': 'de.investments.tools',
  'de.investments.machines.below_250': 'de.investments.machines',
  'de.investments.machines.between_250_and_800': 'de.investments.machines',
  'de.investments.machines.above_800': 'de.investments.machines',
  'de.investments.shop_furniture.below_250': 'de.investments.shop_furniture',
  'de.investments.shop_furniture.between_250_and_800': 'de.investments.shop_furniture',
  'de.investments.shop_furniture.above_800': 'de.investments.shop_furniture'}

In [ ]:
def remapping_function(x):
  if x in category_remap:
    return category_remap[x]
  return x

In [ ]:
#remapping old to new categories
val_df['old_model_category'] = val_df['old_model_category'].apply(remapping_function)

In [ ]:
val_df.to_csv(BASE_PATH / 'Datasets-Versions' / f'{TARGET_COUNTRY}_val.csv')

###Splitting Parsed Df

1. Splitting DF {Train : 75, Test : 12.5, Val : 12.5}
2. Saving DF on Drive

In [ ]:
df = pd.read_csv(BASE_PATH/'Datasets-Versions'/f'{TARGET_COUNTRY}_parsed.csv')

In [ ]:
from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(df, test_size=0.25, random_state=42)

In [ ]:
test_df, val_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [ ]:
train_df.to_csv(BASE_PATH/'Datasets-Versions'/f'{TARGET_COUNTRY}_train.csv')
test_df.to_csv(BASE_PATH/'Datasets-Versions'/f'{TARGET_COUNTRY}_test.csv')
val_df.to_csv(BASE_PATH/'Datasets-Versions'/f'{TARGET_COUNTRY}_val.csv')

###Data Augmentation
(currently not used)

In [ ]:
#GENERATION OF SYNTHETIC DATA

from pydantic import BaseModel
client = OpenAI(api_key=API_KEY)

class Transaction(BaseModel):
    counterparty_name: str
    communication: str
    baseCurrencyAmount: int


def generate_example(label, examples) -> Transaction:
    seed_example = choice(examples)
    prompt = (
        f"Generate a realistic and unique example of a counterparty name for a category with ID '{label}'.\n"
        f"Here is an existing example: {seed_example}\n\n"
        f"New example:"
    )

    response = client.responses.parse(
        model="gpt-4o-mini",
        input = prompt,
        temperature=0.8,
        text_format=Transaction,
    )

    return response.output_parsed

In [ ]:
augmented_rows = []
for label, count in under_represented.items():

    current_examples = df[df['category_id'] == label]['counterparty_name'].tolist()
    needed = 150 - count

    for _ in range(needed):
        fake_transaction = generate_example(label, current_examples)

        augmented_rows.append({
            'counterparty_name': fake_transaction.counterparty_name,
            'communication': fake_transaction.communication,
            'category_id': label,
            'baseCurrencyAmount': fake_transaction.baseCurrencyAmount,
            'cleaned_text_bert:': (fake_transaction.counterparty_name + ' - ' + fake_transaction.communication).lower(),

        })
augmented_df = pd.DataFrame(augmented_rows)

###Training the Model

1. Importing finalized DF
2. Importing define target model and tokenizer
3. Finetuning

In [ ]:
train_df = pd.read_csv(DATASET_PATH / f'{TARGET_COUNTRY}_train.csv')
test_df = pd.read_csv(DATASET_PATH / f'{TARGET_COUNTRY}_test.csv')

In [ ]:
from datetime import datetime
current_date = datetime.now().strftime("%Y-%m-%d")

MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
MODEL_NAME = MODEL_ID.split("/")[-1]
MODEL_NAME_TRAINED = f"{MODEL_NAME}_{TARGET_COUNTRY}_{current_date}"

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

#### Tokenize and metrics

In [ ]:
def tokenize(batch):
    return tokenizer(
        batch["input_text"],
        padding="max_length",
        truncation=True,
        max_length=200
    )

In [ ]:
train_df = train_df.rename(columns={"category_id": "labels"})
test_df = test_df.rename(columns={"category_id": "labels"})

In [ ]:
labels = train_df['labels'].unique().tolist()  # This should give you the string labels
num_labels = len(labels)

# Step 2: Create mappings
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

# Step 3: NOW convert the labels in the DataFrame
train_df['labels'] = train_df['labels'].map(label2id)
test_df["labels"] = test_df["labels"].map(label2id)


# Step 4: Create the dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# tokenize the actual dataset
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/469441 [00:00<?, ? examples/s]

Map:   0%|          | 0/78240 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Metric helper method
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    score = f1_score(labels, predictions, labels=labels, pos_label=1, average="weighted")
    return {"f1": float(score) if score == 1 else score}

#### Finetuning
1. Define model
2. Define TrainingArgs
3. Save Trained Model on Drive

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from huggingface_hub import HfFolder
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, logging

logging.set_verbosity_info()

# Define training args
training_args = TrainingArguments(
    output_dir=MODEL_NAME_TRAINED,
    run_name=MODEL_NAME_TRAINED,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    learning_rate=2e-4,
    num_train_epochs=25,
    bf16=True,
    optim="adamw_torch_fused",
    logging_strategy="steps",
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to=["wandb", "tensorboard"],
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,  # Warmup for 10% of training
    greater_is_better=True,  # Since F1 score should increase
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Stop if no improvement for 3 evals
)
trainer.train()

In [ ]:
trainer.save_model(BASE_PATH / "models" / MODEL_NAME_TRAINED)

Saving model checkpoint to /content/drive/MyDrive/deep-learning/transactions/models/all-MiniLM-L6-v2_de_2025-10-21
Configuration saved in /content/drive/MyDrive/deep-learning/transactions/models/all-MiniLM-L6-v2_de_2025-10-21/config.json
Model weights saved in /content/drive/MyDrive/deep-learning/transactions/models/all-MiniLM-L6-v2_de_2025-10-21/model.safetensors


### Inference
1. Define Pipeline for Inference
2. Run Confusion Matrix

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer

model_path = BASE_PATH / "models" / f'all-MiniLM-L6-v2_{TARGET_COUNTRY}_2025-10-21'
MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

classifier = pipeline(
    task="text-classification",
    model=model_path,
    #model=model,
    tokenizer=tokenizer,
    device=0,
)

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
val_df = pd.read_csv(DATASET_PATH / f'{TARGET_COUNTRY}_val.csv')
val_df = val_df.rename(columns={"category_id": "labels"})
val_dataset = Dataset.from_pandas(val_df)

In [ ]:
val_tokenized_dataset = val_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/78241 [00:00<?, ? examples/s]

In [ ]:
import torch

def classify_batch_pipeline(batch):
    with torch.no_grad():
        classifications = classifier(
            batch['input_text'],
            padding="max_length",
            truncation=True,
            max_length=200
        )
    # Extract the predicted label from the classification output
    predicted_labels = [item['label'] for item in classifications]
    return {'model_prediction': predicted_labels}

# val_tokenized_dataset = val_tokenized_dataset.shuffle().select(range(1000))
# Apply the new classification function using the pipeline
val_tokenized_dataset = val_tokenized_dataset.map(classify_batch_pipeline, batched=True, batch_size=16)

# Convert the dataset to a pandas DataFrame
val_df_pandas = val_tokenized_dataset.to_pandas()

Map:   0%|          | 0/78241 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
# Convert the dataset to a pandas DataFrame
val_df_pandas = val_tokenized_dataset.to_pandas()

In [ ]:
# Now you can use boolean indexing on the pandas DataFrame
correct_predictions_count = val_df_pandas[val_df_pandas['model_prediction'] == val_df_pandas['labels']].shape[0]

print(f"Number of correct predictions: {correct_predictions_count}")

Number of correct predictions: 56211


In [ ]:
# Now you can use boolean indexing on the pandas DataFrame
correct_predictions_count = val_df_pandas[val_df_pandas['model_prediction'] == val_df_pandas['labels']].shape[0]
accuracy = correct_predictions_count/len(val_df_pandas)
print(f"Number of correct predictions: {correct_predictions_count} out of {val_df_pandas.shape[0]}, accuracy {accuracy}")

Number of correct predictions: 59631 out of 78241, accuracy 0.7621451668562519


In [ ]:
# Now you can use boolean indexing on the pandas DataFrame
correct_predictions_count = val_df_pandas[val_df_pandas['model_prediction'] == val_df_pandas['labels']].shape[0]
accuracy = correct_predictions_count/len(val_df_pandas)
print(f"Number of correct predictions: {correct_predictions_count}, accuracy {accuracy}")

In [ ]:
# TODO: use spread with the type above and/or update the type above
val_df_pandas = val_df_pandas.astype({
    "country": "string",
    "APC1": "string",
    "APC2": "string",
    "language": "string",
    "counterparty_name": "string",
    "communication": "string",
    "cleaned_text_bert": "string",
    "model_prediction": "string",
    "VATType": "category",
    "category_description": "category",
    "labels": "category",
    "baseCurrencyAmount": "float64",
    # "old_model_category": "category"
})

In [ ]:
val_df_pandas.to_csv(BASE_PATH / 'Datasets-Versions' / f'{TARGET_COUNTRY}_val_predicted.csv')

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import matplotlib.pyplot as plt

# Get predictions from the model on the evaluation dataset
eval_preds = trainer.predict(val_tokenized_dataset)
predictions = np.argmax(eval_preds.predictions, axis=1)
true_labels = val_tokenized_dataset['labels']
true_labels = [label2id[label] for label in true_labels]

In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(true_labels, predictions)

In [ ]:
# Get the category descriptions corresponding to the true labels
display_labels = [id2label[label] for label in sorted(list(set(true_labels)))]
display_labels = [val_df[val_df['labels'] == label]['category_description'].iloc[0] for label in display_labels]

# Display the confusion matrix
# You might want to adjust the figure size for better readability if you have many labels
fig, ax = plt.subplots(figsize=(100, 100))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=display_labels)
disp.plot(cmap=plt.cm.Blues, ax=ax)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

# Calculate accuracy
accuracy = accuracy_score(val_df_pandas['labels'], val_df_pandas['modern_bert_classification'])
print(f"Accuracy on the evaluation dataset: {accuracy:.4f}")

Accuracy on the evaluation dataset: 0.7465


In [ ]:
# TODO: compute accuracy on old model predictions
accuracy_old = accuracy_score(val_df_pandas['labels'], val_df_pandas['old_model_category'])
print(f"Accuracy of old model on the evaluation dataset: {accuracy_old:.4f}")

Accuracy of old model on the evaluation dataset: 0.3102


### Convert to ONNX

In [ ]:
!pip install optimum

Convert PyTorch classifier models to ONNX format with INT8 quantization for faster inference.

In [ ]:
from optimum.onnxruntime import ORTModelForSequenceClassification, ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig
import os
import shutil

model_path = BASE_PATH / "models" / MODEL_NAME_TRAINED
onnx_path =  BASE_PATH / "models" / MODEL_NAME_TRAINED / '_onnx'

model = ORTModelForSequenceClassification.from_pretrained(
  model_path, export=True, provider="CPUExecutionProvider"
)

# Save the ONNX model (intermediate step)
model.save_pretrained(onnx_path)
quantizer = ORTQuantizer.from_pretrained(onnx_path)

# Use AVX512_VNNI for dynamic INT8 quantization
# This provides strong compression with minimal accuracy loss
dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False)
quantizer.quantize(save_dir=onnx_path, quantization_config=dqconfig)

print(f"✓ Successfully applied INT8 quantization: {onnx_path}")